![Image of mba](https://miro.medium.com/max/1440/1*--iUPe_DtzKdongjqZ2lOg.png)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Change Directory
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/Python/Use-Case/Market_Basket_Analysis/')
os.getcwd()

'/content/drive/My Drive/Colab Notebooks/Python/Use-Case/Customer Clustering'

In [3]:
# Show Data Profile
def show_data_profile(df):
  Data_dict = pd.DataFrame(df.dtypes)
  Data_dict = Data_dict.rename(columns={0: 'Data_Type'})
  Data_dict['Missing'] = df.isnull().sum()
  Data_dict['Missing_Pct'] = round((df.isnull().sum()/df.count())*100, 2)
  Data_dict['Unique'] = df.nunique()
  Data_dict['Count'] = df.count()
  Data_dict=Data_dict.reset_index()
  desc = df.describe(include='all').T
  desc = desc.drop(['count', 'unique'], axis=1)
  desc = desc.reset_index()
  Data_dict2 = pd.merge(Data_dict,desc,how='left', on=['index'])
  Data_dict2 = Data_dict2.fillna('-')#.sort_values(by=['Data_Type','Missing_Pct'],ascending=[True,False])
  Data_dict2 = Data_dict2.reset_index(drop=True).rename(columns={'index':'Features'})
  return Data_dict2

In [4]:
import warnings
warnings.filterwarnings("ignore",category=FutureWarning)
warnings.filterwarnings("ignore",category=RuntimeWarning)


import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('mode.chained_assignment', None)

import re
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns



#DATA PREPARATION

In [5]:
#Load Data
df = pd.read_csv("online-retail-dataset.csv", sep=",",encoding='latin1',dtype={'CustomerID': str} )

In [6]:
df.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


**Attribute Information:**

**InvoiceNo**: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.  

**StockCode**: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.  

**Description**: Product (item) name. Nominal.  

**Quantity**: The quantities of each product (item) per transaction. Numeric.  

**InvoiceDate**: Invice Date and time. Numeric, the day and time when each transaction was generated.  

**UnitPrice**: Unit price. Numeric, Product price per unit in sterling.  

**CustomerID**: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.  

**Country**: Country name. Nominal, the name of the country where each customer resides.

In [7]:
#Rename column replacing whitespace with underscore
df.rename(columns= lambda x : x.replace(' ','_') if ' ' in x else x, inplace=True)
#Remove row with all columns are NA
df = df.dropna(how='all')

In [8]:
show_data_profile(df)

,Features,Data_Type,Missing,Missing_Pct,Unique,Count,top,freq,mean,std,min,25%,50%,75%,max
0,InvoiceNo,object,0,0.00,25900,541909,573585,1114,-,-,-,-,-,-,-
1,StockCode,object,0,0.00,4070,541909,85123A,2313,-,-,-,-,-,-,-
2,Description,object,1454,0.27,4223,540455,WHITE HANGING HEART T-LIGHT HOLDER,2369,-,-,-,-,-,-,-
3,Quantity,int64,0,0.00,722,541909,-,-,9.55225,218.081,-80995,1,3,10,80995
4,InvoiceDate,object,0,0.00,23260,541909,10/31/2011 14:41,1114,-,-,-,-,-,-,-
5,UnitPrice,float64,0,0.00,1630,541909,-,-,4.61111,96.7599,-11062.1,1.25,2.08,4.13,38970
6,CustomerID,object,135080,33.20,4372,406829,17841,7983,-,-,-,-,-,-,-
7,Country,object,0,0.00,38,541909,United Kingdom,495478,-,-,-,-,-,-,-


In [9]:
#Convert datetime column to datetime format
from datetime import datetime
df['InvoiceDate'] = df['InvoiceDate'].apply(lambda x:  datetime.strptime(x, '%m/%d/%Y %H:%M'))

In [10]:
#Split column to alphabet and numeric
df['Status']=df['InvoiceNo'].apply(lambda x : ''.join(re.findall(r"[a-zA-Z]",str(x))))
df['Status']=df['Status'].replace('','S')
df['InvoiceNo']=df['InvoiceNo'].apply(lambda x : ''.join(re.findall(r"\d+",str(x))))

In [11]:
df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Status
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,S
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,S
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,S


In [12]:
df['Status'].unique()

array(['S', 'C', 'A'], dtype=object)

In [13]:
dF = df[df['Status']=='S']

## Data Cleaning

In [14]:
#Remove row with CustomerID=NA
df = df[~df['CustomerID'].isna()]
df['Description'] = df['Description'].fillna('')

In [15]:
#Remove row with Negative Quantity
df = df[df['Quantity'] >0]

In [16]:
show_data_profile(df)

,Features,Data_Type,Missing,Missing_Pct,Unique,Count,top,freq,first,last,mean,std,min,25%,50%,75%,max
0,InvoiceNo,object,0,0.0,18536,397924,576339,542,-,-,-,-,-,-,-,-,-
1,StockCode,object,0,0.0,3665,397924,85123A,2035,-,-,-,-,-,-,-,-,-
2,Description,object,0,0.0,3877,397924,WHITE HANGING HEART T-LIGHT HOLDER,2028,-,-,-,-,-,-,-,-,-
3,Quantity,int64,0,0.0,302,397924,-,-,-,-,13.0218,180.42,1,2,6,12,80995
4,InvoiceDate,datetime64[ns],0,0.0,17286,397924,2011-11-14 15:27:00,542,2010-12-01 08:26:00,2011-12-09 12:50:00,-,-,-,-,-,-,-
5,UnitPrice,float64,0,0.0,441,397924,-,-,-,-,3.11617,22.0968,0,1.25,1.95,3.75,8142.75
6,CustomerID,object,0,0.0,4339,397924,17841,7847,-,-,-,-,-,-,-,-,-
7,Country,object,0,0.0,37,397924,United Kingdom,354345,-,-,-,-,-,-,-,-,-
8,Status,object,0,0.0,1,397924,S,397924,-,-,-,-,-,-,-,-,-


## Filter Only France

In [17]:
df = df[df['Country'] =="France"]

# CREATE BASKET

In [29]:
#Count items quantity
df_sumquantity = df.groupby(['InvoiceNo', 'Description'])[['Quantity']].sum().reset_index()
df_sumquantity

,InvoiceNo,Description,Quantity
0,536370,SET 2 TEA TOWELS I LOVE LONDON,24
1,536370,ALARM CLOCK BAKELIKE GREEN,12
2,536370,ALARM CLOCK BAKELIKE PINK,24
3,536370,ALARM CLOCK BAKELIKE RED,24
4,536370,CHARLOTTE BAG DOLLY GIRL DESIGN,20
...,...,...,...
8301,581587,CIRCUS PARADE LUNCH BOX,12
8302,581587,PACK OF 20 SPACEBOY NAPKINS,12
8303,581587,PLASTERS IN TIN CIRCUS PARADE,12
8304,581587,PLASTERS IN TIN STRONGMAN,12


In [30]:
#Pivot table
basket = df_sumquantity.pivot_table(values='Quantity', index='InvoiceNo', columns='Description')

In [31]:
#Fillna
basket = basket.fillna(0)
basket

Description,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,15CM CHRISTMAS GLASS BALL 20 LIGHTS,16 PIECE CUTLERY SET PANTRY DESIGN,18PC WOODEN CUTLERY SET DISPOSABLE,20 DOLLY PEGS RETROSPOT,200 RED + WHITE BENDY STRAWS,3 HOOK HANGER MAGIC GARDEN,3 PIECE SPACEBOY COOKIE CUTTER SET,3 RAFFIA RIBBONS 50'S CHRISTMAS,3 STRIPEY MICE FELTCRAFT,3 TIER CAKE TIN RED AND CREAM,3 TRADITIONAl BISCUIT CUTTERS SET,36 DOILIES DOLLY GIRL,36 DOILIES VINTAGE CHRISTMAS,36 FOIL HEART CAKE CASES,36 FOIL STAR CAKE CASES,36 PENCILS TUBE RED RETROSPOT,36 PENCILS TUBE SKULLS,36 PENCILS TUBE WOODLAND,3D HEARTS HONEYCOMB PAPER GARLAND,3D TRADITIONAL CHRISTMAS STICKERS,3D VINTAGE CHRISTMAS STICKERS,4 IVORY DINNER CANDLES SILVER FLOCK,4 PINK DINNER CANDLE SILVER FLOCK,4 TRADITIONAL SPINNING TOPS,5 HOOK HANGER MAGIC TOADSTOOL,5 HOOK HANGER RED MAGIC TOADSTOOL,6 GIFT TAGS 50'S CHRISTMAS,6 GIFT TAGS VINTAGE CHRISTMAS,6 RIBBONS EMPIRE,6 RIBBONS RUSTIC CHARM,6 ROCKET BALLOONS,60 CAKE CASES DOLLY GIRL DESIGN,60 CAKE CASES VINTAGE CHRISTMAS,60 TEATIME FAIRY CAKE CASES,6PC WOOD PLATE SET DISPOSABLE,72 SWEETHEART FAIRY CAKE CASES,A PRETTY THANK YOU CARD,ABC TREASURE BOOK BOX,ADULT APRON APPLE DELIGHT,ADVENT CALENDAR GINGHAM SACK,AGED GLASS SILVER T-LIGHT HOLDER,AIRLINE BAG VINTAGE JET SET BROWN,AIRLINE BAG VINTAGE JET SET RED,AIRLINE BAG VINTAGE JET SET WHITE,AIRLINE BAG VINTAGE TOKYO 78,AIRLINE BAG VINTAGE WORLD CHAMPION,"AIRLINE LOUNGE,METAL SIGN",ALARM CLOCK BAKELIKE CHOCOLATE,ALARM CLOCK BAKELIKE GREEN,ALARM CLOCK BAKELIKE IVORY,ALARM CLOCK BAKELIKE ORANGE,ALARM CLOCK BAKELIKE PINK,ALARM CLOCK BAKELIKE RED,ALPHABET HEARTS STICKER SHEET,ALUMINIUM STAMPED HEART,AMETHYST CHUNKY BEAD BRACELET W STR,ANGEL DECORATION STARS ON DRESS,ANTIQUE ALL GLASS CANDLESTICK,ANTIQUE GLASS DRESSING TABLE POT,ANTIQUE GLASS PEDESTAL BOWL,ANTIQUE SILVER BAUBLE LAMP,ANTIQUE SILVER T-LIGHT GLASS,ANTIQUE SILVER TEA GLASS ENGRAVED,APPLE BATH SPONGE,AREA PATROLLED METAL SIGN,ASS COLOUR GLOWING TIARAS,ASS FLORAL PRINT MULTI SCREWDRIVER,ASSORTED BOTTLE TOP MAGNETS,ASSORTED COLOUR BIRD ORNAMENT,ASSORTED COLOUR MINI CASES,ASSORTED COLOUR T-LIGHT HOLDER,ASSORTED COLOURS SILK FAN,ASSORTED EASTER DECORATIONS BELLS,ASSORTED EASTER GIFT TAGS,"ASSORTED FLOWER COLOUR ""LEIS""",ASSORTED TUTTI FRUTTI BRACELET,ASSORTED TUTTI FRUTTI MIRROR,ASSORTED TUTTI FRUTTI SMALL PURSE,ASSTD DESIGN 3D PAPER STICKERS,ASSTD DESIGN RACING CAR PEN,ASSTD FRUIT+FLOWERS FRIDGE MAGNETS,ASSTD RASTA KEY-CHAINS,BABUSHKA LIGHTS STRING OF 10,BAG 125g SWIRLY MARBLES,BAG 250g SWIRLY MARBLES,BAKING MOULD CHOCOLATE CUPCAKES,BAKING MOULD EASTER EGG MILK CHOC,BAKING MOULD EASTER EGG WHITE CHOC,BAKING MOULD HEART MILK CHOCOLATE,BAKING MOULD HEART WHITE CHOCOLATE,BAKING SET 9 PIECE RETROSPOT,BAKING SET SPACEBOY DESIGN,BALLOON ART MAKE YOUR OWN FLOWERS,BALLOON PUMP WITH 10 BALLOONS,BALLOON WATER BOMB PACK OF 35,BALLOONS WRITING SET,BANQUET BIRTHDAY CARD,BASKET OF TOADSTOOLS,BATH BUILDING BLOCK WORD,BEADED CRYSTAL HEART BLUE LARGE,BELLE JARDINIERE CUSHION COVER,BEWARE OF THE CAT METAL SIGN,BIG DOUGHNUT FRIDGE MAGNETS,BILI NUT AND WOOD NECKLACE,BINGO SET,BIRD DECORATION GREEN POLKADOT,BIRD DECORATION RED RETROSPOT,BIRD HOUSE HOT WATER BOTTLE,BIRDS MOBILE VINTAGE DESIGN,"BIRTHDAY CARD, RETRO SPOT",BISCUIT TIN 50'S CHRISTMAS,BISCUIT TIN VINTAGE CHRISTMAS,BISCUIT TIN VINTAGE GREEN,BISCUIT TIN VINTAGE RED,BLACK AND WHITE CAT BOWL,BLACK CANDELABRA T-LIGHT HOLDER,BLACK CHRISTMAS TREE 120CM,BLACK CHUNKY BEAD BRACELET W STRAP,BLACK ENAMEL FLOWER RING,BLACK KITCHEN SCALES,BLACK LOVE BIRD CANDLE,BLACK PIRATE TREASURE CHEST,BLACK RECORD COVER 

In [32]:
#Convert basket Values : if value <=0 then value=0 elif value > 0 then value = 1
basket_sets = basket.applymap(lambda x : 0 if x<= 0 else 1)
basket_sets

Description,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,15CM CHRISTMAS GLASS BALL 20 LIGHTS,16 PIECE CUTLERY SET PANTRY DESIGN,18PC WOODEN CUTLERY SET DISPOSABLE,20 DOLLY PEGS RETROSPOT,200 RED + WHITE BENDY STRAWS,3 HOOK HANGER MAGIC GARDEN,3 PIECE SPACEBOY COOKIE CUTTER SET,3 RAFFIA RIBBONS 50'S CHRISTMAS,3 STRIPEY MICE FELTCRAFT,3 TIER CAKE TIN RED AND CREAM,3 TRADITIONAl BISCUIT CUTTERS SET,36 DOILIES DOLLY GIRL,36 DOILIES VINTAGE CHRISTMAS,36 FOIL HEART CAKE CASES,36 FOIL STAR CAKE CASES,36 PENCILS TUBE RED RETROSPOT,36 PENCILS TUBE SKULLS,36 PENCILS TUBE WOODLAND,3D HEARTS HONEYCOMB PAPER GARLAND,3D TRADITIONAL CHRISTMAS STICKERS,3D VINTAGE CHRISTMAS STICKERS,4 IVORY DINNER CANDLES SILVER FLOCK,4 PINK DINNER CANDLE SILVER FLOCK,4 TRADITIONAL SPINNING TOPS,5 HOOK HANGER MAGIC TOADSTOOL,5 HOOK HANGER RED MAGIC TOADSTOOL,6 GIFT TAGS 50'S CHRISTMAS,6 GIFT TAGS VINTAGE CHRISTMAS,6 RIBBONS EMPIRE,6 RIBBONS RUSTIC CHARM,6 ROCKET BALLOONS,60 CAKE CASES DOLLY GIRL DESIGN,60 CAKE CASES VINTAGE CHRISTMAS,60 TEATIME FAIRY CAKE CASES,6PC WOOD PLATE SET DISPOSABLE,72 SWEETHEART FAIRY CAKE CASES,A PRETTY THANK YOU CARD,ABC TREASURE BOOK BOX,ADULT APRON APPLE DELIGHT,ADVENT CALENDAR GINGHAM SACK,AGED GLASS SILVER T-LIGHT HOLDER,AIRLINE BAG VINTAGE JET SET BROWN,AIRLINE BAG VINTAGE JET SET RED,AIRLINE BAG VINTAGE JET SET WHITE,AIRLINE BAG VINTAGE TOKYO 78,AIRLINE BAG VINTAGE WORLD CHAMPION,"AIRLINE LOUNGE,METAL SIGN",ALARM CLOCK BAKELIKE CHOCOLATE,ALARM CLOCK BAKELIKE GREEN,ALARM CLOCK BAKELIKE IVORY,ALARM CLOCK BAKELIKE ORANGE,ALARM CLOCK BAKELIKE PINK,ALARM CLOCK BAKELIKE RED,ALPHABET HEARTS STICKER SHEET,ALUMINIUM STAMPED HEART,AMETHYST CHUNKY BEAD BRACELET W STR,ANGEL DECORATION STARS ON DRESS,ANTIQUE ALL GLASS CANDLESTICK,ANTIQUE GLASS DRESSING TABLE POT,ANTIQUE GLASS PEDESTAL BOWL,ANTIQUE SILVER BAUBLE LAMP,ANTIQUE SILVER T-LIGHT GLASS,ANTIQUE SILVER TEA GLASS ENGRAVED,APPLE BATH SPONGE,AREA PATROLLED METAL SIGN,ASS COLOUR GLOWING TIARAS,ASS FLORAL PRINT MULTI SCREWDRIVER,ASSORTED BOTTLE TOP MAGNETS,ASSORTED COLOUR BIRD ORNAMENT,ASSORTED COLOUR MINI CASES,ASSORTED COLOUR T-LIGHT HOLDER,ASSORTED COLOURS SILK FAN,ASSORTED EASTER DECORATIONS BELLS,ASSORTED EASTER GIFT TAGS,"ASSORTED FLOWER COLOUR ""LEIS""",ASSORTED TUTTI FRUTTI BRACELET,ASSORTED TUTTI FRUTTI MIRROR,ASSORTED TUTTI FRUTTI SMALL PURSE,ASSTD DESIGN 3D PAPER STICKERS,ASSTD DESIGN RACING CAR PEN,ASSTD FRUIT+FLOWERS FRIDGE MAGNETS,ASSTD RASTA KEY-CHAINS,BABUSHKA LIGHTS STRING OF 10,BAG 125g SWIRLY MARBLES,BAG 250g SWIRLY MARBLES,BAKING MOULD CHOCOLATE CUPCAKES,BAKING MOULD EASTER EGG MILK CHOC,BAKING MOULD EASTER EGG WHITE CHOC,BAKING MOULD HEART MILK CHOCOLATE,BAKING MOULD HEART WHITE CHOCOLATE,BAKING SET 9 PIECE RETROSPOT,BAKING SET SPACEBOY DESIGN,BALLOON ART MAKE YOUR OWN FLOWERS,BALLOON PUMP WITH 10 BALLOONS,BALLOON WATER BOMB PACK OF 35,BALLOONS WRITING SET,BANQUET BIRTHDAY CARD,BASKET OF TOADSTOOLS,BATH BUILDING BLOCK WORD,BEADED CRYSTAL HEART BLUE LARGE,BELLE JARDINIERE CUSHION COVER,BEWARE OF THE CAT METAL SIGN,BIG DOUGHNUT FRIDGE MAGNETS,BILI NUT AND WOOD NECKLACE,BINGO SET,BIRD DECORATION GREEN POLKADOT,BIRD DECORATION RED RETROSPOT,BIRD HOUSE HOT WATER BOTTLE,BIRDS MOBILE VINTAGE DESIGN,"BIRTHDAY CARD, RETRO SPOT",BISCUIT TIN 50'S CHRISTMAS,BISCUIT TIN VINTAGE CHRISTMAS,BISCUIT TIN VINTAGE GREEN,BISCUIT TIN VINTAGE RED,BLACK AND WHITE CAT BOWL,BLACK CANDELABRA T-LIGHT HOLDER,BLACK CHRISTMAS TREE 120CM,BLACK CHUNKY BEAD BRACELET W STRAP,BLACK ENAMEL FLOWER RING,BLACK KITCHEN SCALES,BLACK LOVE BIRD CANDLE,BLACK PIRATE TREASURE CHEST,BLACK RECORD COVER 

# CALCULATE APRIORI

**Apriori** algorithm is simply used to find the frequently bought items in the dataset  

**Support {freq (x) / n, range: [0, 1]}** gives the fraction of transactions which contains item x. It tells us about the frequently bought items and we can filter out the items that have a low frequency.  

In [34]:
from mlxtend.frequent_patterns import apriori

In [35]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)
frequent_itemsets.head()

,support,itemsets
0,0.071979,(4 TRADITIONAL SPINNING TOPS)
1,0.097686,(ALARM CLOCK BAKELIKE GREEN)
2,0.102828,(ALARM CLOCK BAKELIKE PINK)
3,0.095116,(ALARM CLOCK BAKELIKE RED )
4,0.077121,(BAKING SET 9 PIECE RETROSPOT )


# CALCULATE ASSOCIATION RULE

**Support {freq (x,y) / n, range: [0, 1]}** gives the fraction of transactions which contains item x and y. It tells us about the frequently bought items or the combination of items bought frequently and we can filter out the items that have a low frequency.  

**Confidence {freq(x,y) / freq (x), range: [0, 1]}** tells us how often the items x and y occur together, given the number times x occurs.  

**Lift {support/support(x) * support(y), range: [0, inf]}** indicates the strength of a rule over the random occurrence of x and y. It explains the strength of a rule and more the Lift more is the strength.

In [36]:
from mlxtend.frequent_patterns import association_rules

In [37]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.097686,0.102828,0.074550,0.763158,7.421711,0.064505,3.788061
1,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.102828,0.097686,0.074550,0.725000,7.421711,0.064505,3.281140
2,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED ),0.097686,0.095116,0.079692,0.815789,8.576814,0.070400,4.912229
3,(ALARM CLOCK BAKELIKE RED ),(ALARM CLOCK BAKELIKE GREEN),0.095116,0.097686,0.079692,0.837838,8.576814,0.070400,5.564267
4,(POSTAGE),(ALARM CLOCK BAKELIKE GREEN),0.771208,0.097686,0.084833,0.110000,1.126053,0.009496,1.013836
...,...,...,...,...,...,...,...,...,...
133,"(SET/6 RED SPOTTY PAPER CUPS, SET/6 RED SPOTTY...","(SET/20 RED RETROSPOT PAPER NAPKINS , POSTAGE)",0.123393,0.110540,0.082262,0.666667,6.031008,0.068622,2.668380
134,(SET/20 RED RETROSPOT PAPER NAPKINS ),"(POSTAGE, SET/6 RED SPOTTY PAPER CUPS, SET/6 R...",0.133676,0.102828,0.082262,0.615385,5.984615,0.068517,2.332648
135,(POSTAGE),"(SET/20 RED RETROSPOT PAPER NAPKINS , SET/6 RE...",0.771208,0.100257,0.082262,0.106667,1.063932,0.004943,1.007175
136,(SET/6 RED SPOTTY PAPER CUPS),"(SET/20 RED RETROSPOT PAPER NAPKINS , POSTAGE,...",0.138817,0.084833,0.082262,0.592593,6.985410,0.070486,2.246319


## Filter by confidence and lift

In [38]:
#melakukan filter untuk nilai lift ratio minimal 6 dan tingkat confidence minimal 0.8
rules[(rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED ),0.097686,0.095116,0.079692,0.815789,8.576814,0.070400,4.912229
3,(ALARM CLOCK BAKELIKE RED ),(ALARM CLOCK BAKELIKE GREEN),0.095116,0.097686,0.079692,0.837838,8.576814,0.070400,5.564267
74,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.138817,0.128535,0.123393,0.888889,6.915556,0.105550,7.843188
75,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.128535,0.138817,0.123393,0.960000,6.915556,0.105550,21.529563
76,"(POSTAGE, ALARM CLOCK BAKELIKE GREEN)",(ALARM CLOCK BAKELIKE RED ),0.084833,0.095116,0.071979,0.848485,8.920557,0.063910,5.972237
77,"(POSTAGE, ALARM CLOCK BAKELIKE RED )",(ALARM CLOCK BAKELIKE GREEN),0.087404,0.097686,0.071979,0.823529,8.430341,0.063441,5.113111
112,"(POSTAGE, SET/6 RED SPOTTY PAPER CUPS)",(SET/6 RED SPOTTY PAPER PLATES),0.118252,0.128535,0.102828,0.869565,6.765217,0.087628,6.681234
113,"(POSTAGE, SET/6 RED SPOTTY PAPER PLATES)",(SET/6 RED SPOTTY PAPER CUPS),0.107969,0.138817,0.102828,0.952381,6.860670,0.087840,18.084833
117,(SET/6 RED SPOTTY PAPER PLATES),"(POSTAGE, SET/6 RED SPOTTY PAPER CUPS)",0.128535,0.118252,0.102828,0.800000,6.765217,0.087628,4.408740
118,"(SET/20 RED RETROSPOT PAPER NAPKINS , SET/6 RE...",(SET/6 RED SPOTTY PAPER PLATES),0.102828,0.128535,0.100257,0.975000,7.585500,0.087040,34.858612
